## New York City Yellow Taxi Trip Data in First Quarter of 2019

In [4]:
import numpy as np
import pandas as pd

from datetime import datetime
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split

## Read Data

In [5]:
# 2019 Yellow Taxi Trip Data
def extract_data(url, query):
    new_url = url + query
    df = pd.DataFrame()
    
    i = 0
    cur_page = new_url + "&$limit=1000&$offset=" + str(i)
    cur_data = pd.read_json(cur_page)
    while len(cur_data) >= 1000:
        df = df.append(cur_data, sort=True)
        i += 1000
        cur_page = new_url + "&$limit=1000&$offset=" + str(i)
        cur_data = pd.read_json(cur_page)
        
        if i % 4e5 == 0:
            print("{} records have been extracted at {}.".format(i, datetime.now().time()))
    
    if len(cur_data) > 0:
        df = df.append(cur_data, sort=True)
    
    return df

In [6]:
def save_to_csv(df, fileName="./data/myData.csv"):
    df.to_csv(fileName)

In [7]:
def get_data():
    url = "https://data.cityofnewyork.us/resource/2upf-qytp.json?"
    # pickup time: Jan. ~ June, pickup location: <= 50
    query = "$where=tpep_pickup_datetime between '2019-01-01' and '2019-04-01' and PULocationID<=50"
    query = query.replace(" ", "%20")
    print(url + query)  # first page

    print("Start time: ", datetime.now().time())
    df = extract_data(url, query)
    print("End time: ", datetime.now().time())

    save_to_csv(df)

In [8]:
def read_from_csv(path):
    df = pd.read_csv(path, index_col=0)
    return df

## Data Description

| **Attribute** | **Taxi Information** | 
|----------|:-------------|
| VendorID | A code indicating the LPEP provider that provided the record. (1= Creative Mobile Technologies, LLC 2= VeriFone Inc.)|
| tpep_pickup_datetime | The date and time when the meter was engaged.|
| tpep_dropoff_datetime | The date and time when the meter was disengaged.|
| Passenger_count| The number of passengers in the vehicle.|
| Trip_distance | The elapsed trip distance in miles reported by the taximeter.|
| PULocationID | TLC Taxi Zone in which the taximeter was engaged|
| DOLocationID | TLC Taxi Zone in which the taximeter was disengaged|
| RateCodeID | The final rate code in effect at the end of the trip.(1= Standard rate 2=JFK 3=Newark 4=Nassau or Westchester 5=Negotiated fare 6=Group ride)|  
| Payment_type| A numeric code signifying how the passenger paid for the trip.(1= Credit card 2= Cash 3= No charge 4= Dispute 5= Unknown 6= Voided trip)|
| Fare_amount | The time-and-distance fare calculated by the meter.|
|Extra | Miscellaneous extras and surcharges. Currently, this only includes the \$0.50 and \$1.0 rush hour and overnight charges.|
| MTA_tax | \$0.50 MTA tax that is automatically triggered based on the metered rate in use.|
| Improvement_surcharge | \$0.30 improvement surcharge assessed on hailed trips at the flag drop.|
| Tip amount | This field is automatically populated for credit card tips. Cash tips are not included.|
| Tolls_amount | Total amount of all tolls paid in trip.|
| Total_amount | The total amount charged to passengers. Does not include cash tips.|




## Data Cleaning

In [9]:
def drop_useless_columns(df):
    # get rid of useless columns
    df = df.drop(columns=["congestion_surcharge", "store_and_fwd_flag"])
    return df

In [10]:
def convert_to_timestamp(df):
    df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
    df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])
    return df

In [11]:
def create_duration_colum(df):
    # create "duration" column from pickup and dropoff datetime
    df["duration"] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
    # convert to minutes
    df["duration"] = df['duration'].dt.total_seconds() / 60
    return df

In [12]:
path = "./data/myData.csv"
df = read_from_csv(path)
df = drop_useless_columns(df)
df = convert_to_timestamp(df)
df = create_duration_colum(df)

df.columns

/usr/local/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Index(['dolocationid', 'extra', 'fare_amount', 'improvement_surcharge',
       'mta_tax', 'passenger_count', 'payment_type', 'pulocationid',
       'ratecodeid', 'tip_amount', 'tolls_amount', 'total_amount',
       'tpep_dropoff_datetime', 'tpep_pickup_datetime', 'trip_distance',
       'vendorid', 'duration'],
      dtype='object')

In [13]:
df.head(10)

,dolocationid,extra,fare_amount,improvement_surcharge,mta_tax,passenger_count,payment_type,pulocationid,ratecodeid,tip_amount,tolls_amount,total_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,vendorid,duration
0,1,0.0,20.0,0.3,0.5,1,1,1,5,1.00,0.00,21.80,2019-01-01 02:19:32,2019-01-01 02:19:27,0.0,2,0.083333
1,1,0.0,85.0,0.3,0.0,1,1,1,5,2.00,0.00,87.30,2019-01-01 05:52:53,2019-01-01 05:52:45,0.0,2,0.133333
2,1,0.0,80.0,0.3,0.0,1,1,1,5,0.00,0.00,80.30,2019-01-01 06:23:23,2019-01-01 06:22:24,0.0,2,0.983333
3,1,0.0,80.0,0.3,0.0,2,1,1,5,16.06,0.00,96.36,2019-01-01 08:13:01,2019-01-01 08:12:51,0.0,2,0.166667
4,1,0.0,160.0,0.3,0.5,2,1,1,5,0.00,0.00,160.80,2019-01-01 08:14:53,2019-01-01 08:14:48,0.0,2,0.083333
5,1,0.0,2.5,0.3,0.5,3,2,1,1,0.00,0.00,3.30,2019-01-01 10:10:33,2019-01-01 10:10:30,16.9,1,0.050000
6,1,0.0,0.0,0.3,0.0,3,3,1,5,0.00,16.26,16.56,2019-01-01 10:11:32,2019-01-01 10:10:52,16.9,1,0.666667
7,1,0.0,0.0,0.3,0.0,3,3,1,5,0.00,16.26,16.56,2019-01-01 10:13:13,2019-01-01 10:12:05,16.9,1,1.133333
8,1,0.0,120.0,0.3,0.0,3,2,1,5,0.00,16.26,136.56,2019-01-01 10:14:35,2019-01-01 10:13:44,16.9,1,0.850000
9,1,0.0,90.0,0.3,0.0,3,1,1,5,18.06,0.00,108.36,2019-01-01 12:16:23,2019-01-01 12:16:18,0.0,2,0.083333


In [14]:
len(df)

1838913

In [15]:
def drop_outliers(df):
    # drop rows with 0.0 total amount or 0.0 trip distance
    df = df[(df['total_amount'] > 0.0) & (df['trip_distance'] > 0.0)]
    
    # trip duration should be less than or equal to 40 minutes
    df = df[df['duration'] <= 90]
    
    # count of passengers should be positive and less than 5
    df = df[(df['passenger_count'] > 0) & (df['passenger_count'] <= 4)]
    
    # trip distance between 1 and 20 miles
    df = df[(df['trip_distance'] >= 1.) & (df['trip_distance'] <= 20)]
    
    # only reserve standard code trip
    df = df[df['ratecodeid'] == 1]
    
    # get rid of payment type that is "dispute" or "no charge"
    df = df[(df['payment_type'] == 1) | (df['payment_type'] == 2)]
    
    df = df.dropna().reset_index(drop=True)
    return df

In [16]:
df = drop_outliers(df)

len(df)

1196085

In [17]:
df.head(10)

,dolocationid,extra,fare_amount,improvement_surcharge,mta_tax,passenger_count,payment_type,pulocationid,ratecodeid,tip_amount,tolls_amount,total_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,vendorid,duration
0,1,0.0,2.5,0.3,0.5,3,2,1,1,0.00,0.0,3.30,2019-01-01 10:10:33,2019-01-01 10:10:30,16.90,1,0.050000
1,265,0.0,24.5,0.3,0.5,1,2,1,1,0.00,0.0,25.30,2019-01-02 15:11:48,2019-01-02 14:50:55,7.92,2,20.883333
2,249,1.0,41.0,0.3,0.5,1,2,1,1,0.00,17.5,60.30,2019-01-09 17:23:39,2019-01-09 16:55:48,14.97,2,27.850000
3,237,1.0,8.5,0.3,0.5,1,1,1,1,1.00,0.0,11.30,2019-01-09 19:23:18,2019-01-09 19:13:21,2.01,2,9.950000
4,265,0.0,7.5,0.3,0.5,1,1,1,1,1.66,0.0,9.96,2019-01-11 07:52:27,2019-01-11 07:43:43,1.04,2,8.733333
5,1,1.0,2.5,0.3,0.5,4,1,1,1,39.00,0.0,43.30,2019-01-11 17:37:53,2019-01-11 17:37:51,14.60,1,0.033333
6,1,0.0,13.0,0.3,0.5,1,2,1,1,0.00,0.0,13.80,2019-01-16 15:11:33,2019-01-16 14:59:21,3.31,2,12.200000
7,265,0.0,38.5,0.3,0.5,1,2,1,1,0.00,0.0,39.30,2019-01-22 10:10:52,2019-01-22 09:32:00,11.20,1,38.866667
8,1,1.0,2.5,0.3,0.5,1,2,1,1,0.00,0.0,4.30,2019-01-25 16:35:23,2019-01-25 16:35:18,17.40,1,0.083333
9,1,1.0,2.5,0.3,0.5,1,2,1,1,0.00,0.0,4.30,2019-01-25 16:37:27,2019-01-25 16:37:23,17.40,1,0.066667


In [18]:
# save_to_csv(df, "./data/processed_myData.csv")

## Data Split

In [19]:
X_train, X_test = train_test_split(df, test_size=0.2)
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

print("Train size: {}".format(len(X_train)))
print("Test size: {}".format(len(X_test)))

Train size: 956868
Test size: 239217


In [20]:
save_to_csv(X_train, "./data/train.csv")
save_to_csv(X_test, "./data/test.csv")